In [149]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import zero_one_loss
import matplotlib.pyplot as plt

# Load datasets

### Breast Cancer

In [99]:
dataset = load_breast_cancer()

In [100]:
df_x_bc = pd.DataFrame(dataset['data'], columns = dataset['feature_names']) #samples (539 vectors)

In [101]:
df_y_bc = dataset['target'] # labels (539 labels 1 == maniglent, 0 == benign)

### Phoneme

In [291]:
df_phoneme = pd.read_csv("./datasets/phoneme.dat/data", sep="\s+")

In [292]:
df_x_phoneme = df_phoneme.iloc[:,0:5]

In [293]:
df_y_phoneme = df_phoneme.iloc[:,5:6]

# Initialize Classifiers

In [335]:
n_estimators = 1000 # boosting iterations
max_depth = 8
number_of_split_rounds = 100

In [77]:
rf = RandomForestClassifier(max_depth=max_depth, n_estimators=number_of_estimators) # init RandomForests

# Run Classifiers

In [123]:
# for r in range(number_of_split_rounds):
#     X_train, X_test, Y_train, Y_test = train_test_split(df_x_bc, df_y_bc, test_size=0.3)
#     ada.fit(X_train, Y_train)
#     rf.fit(X_train, Y_train)

In [336]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=max_depth),
    n_estimators=n_estimators,
    algorithm="SAMME.R") # init AdaBoost

In [337]:
X_train, X_test, Y_train, Y_test = train_test_split(df_x_phoneme, df_y_phoneme, test_size=0.3)

In [ ]:
ada.fit(X_train, Y_train);

/home/emilio/Software/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
ada_err_test = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada.staged_predict(X_test)):
    ada_err_test[i] = zero_one_loss(y_pred, Y_test)

ada_err_train = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada.staged_predict(X_train)):
    ada_err_train[i] = zero_one_loss(y_pred, Y_train)

In [ ]:
plt.figure()
plt.plot(ada_err_train, label='Training error')
plt.plot(ada_err_test, label='Test error')
plt.ylabel('error rate')
plt.xlabel('boosting interations')
plt.legend()
plt.show()